In [6]:
import pandas as pd
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Dropout, Masking, Embedding
from matplotlib import pyplot as plt
from ipywidgets import interact
plt.rcParams['figure.dpi'] = 200

In [7]:
file_headers = ["/media/user1/Data 2000/soft_robotics_experiments/training_data/round_1/module1_fullext1",
                "/media/user1/Data 2000/soft_robotics_experiments/training_data/round_1/module1_fullext2",
                "/media/user1/Data 2000/soft_robotics_experiments/training_data/round_1/module1_fullext3",
                "/media/user1/Data 2000/soft_robotics_experiments/training_data/round_1/module1_fullext4",
                "/media/user1/Data 2000/soft_robotics_experiments/training_data/round_1/module2_fullext1",
                "/media/user1/Data 2000/soft_robotics_experiments/training_data/round_1/module2_fullext2",
                "/media/user1/Data 2000/soft_robotics_experiments/training_data/round_1/module2_fullext3",
                "/media/user1/Data 2000/soft_robotics_experiments/training_data/round_1/module2_fullext4",
                "/media/user1/Data 2000/soft_robotics_experiments/training_data/round_1/s_curve1",
                "/media/user1/Data 2000/soft_robotics_experiments/training_data/round_1/s_curve2"]

CSV_SFX = ".csv"
MARKERS_SFX = "_markers"
POLY_SFX = "_poly"
M10Y = "M10Y"

In [8]:
# distance goes from 22-38. 16 cm of extension at least which is pretty good.

In [15]:
poly_df_list = []
data_df_list = []

for header in file_headers:
    data_file = header + CSV_SFX
    marker_file = header + MARKERS_SFX + CSV_SFX
    poly_file = header + POLY_SFX + CSV_SFX
    
    data_df = pd.read_csv(data_file)
    marker_df = pd.read_csv(marker_file)
    poly_df = pd.read_csv(poly_file)
    
    zero_indices = list(marker_df[marker_df[M10Y] == 0].index)
    
    data_df = data_df.drop(data_df.index[zero_indices]).reset_index()
    poly_df = poly_df.drop(poly_df.index[zero_indices])
    
    data_df_list.append(data_df)
    poly_df_list.append(poly_df)

total_data_df = pd.concat(data_df_list).reset_index()
total_poly_df = pd.concat(poly_df_list).reset_index()

training_data_list = [total_data_df, total_poly_df]
all_df = pd.concat(training_data_list, axis=1)

x_labels = ["M1-PL", "M1-PR", "M2-PL", "M2-PR"]
y_labels = ["a0", "a1", "a2", "a3", "a4", "a5", "d"]

def normalize_columns(df, col_list):
    min_vals = []
    max_vals = []
    for c in col_list:
        min_val = df[c].min()
        max_val = df[c].max()
        min_vals.append(min_val)
        max_vals.append(max_val)
        df[c] = (df[c] - min_val) / (max_val - min_val)
    return min_vals, max_vals
        
x_mins, x_maxes = normalize_columns(all_df, x_labels)
y_mins, y_maxes = normalize_columns(all_df, y_labels)

# print(x_mins, x_maxes)

# shuffle the rows
train_df = all_df.sample(frac=0.8, random_state=25)
test_df = all_df.drop(train_df.index)

x_dim = len(x_labels)
y_dim = len(y_labels)

x_train = train_df[x_labels]
y_train = train_df[y_labels]

x_test = test_df[x_labels]
y_test = test_df[y_labels]

In [16]:
model = Sequential()
model.add(Dense(16, input_dim=x_dim, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(16, input_dim=16, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(8, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(y_dim, activation='linear'))
model.compile(loss='mse', optimizer='adam', metrics=['accuracy'])

In [17]:
model.fit(x_train, y_train, epochs=300, batch_size=10)

Epoch 1/300
490/490 [==============================] - 0s 384us/step - loss: 0.1508 - accuracy: 0.1974
Epoch 2/300
490/490 [==============================] - 0s 381us/step - loss: 0.0287 - accuracy: 0.5386
Epoch 3/300
490/490 [==============================] - 0s 383us/step - loss: 0.0215 - accuracy: 0.6189
Epoch 4/300
490/490 [==============================] - 0s 381us/step - loss: 0.0160 - accuracy: 0.6464
Epoch 5/300
490/490 [==============================] - 0s 389us/step - loss: 0.0132 - accuracy: 0.7097
Epoch 6/300
490/490 [==============================] - 0s 382us/step - loss: 0.0117 - accuracy: 0.7276
Epoch 7/300
490/490 [==============================] - 0s 383us/step - loss: 0.0110 - accuracy: 0.7285
Epoch 8/300
490/490 [==============================] - 0s 387us/step - loss: 0.0103 - accuracy: 0.7373
Epoch 9/300
490/490 [==============================] - 0s 392us/step - loss: 0.0098 - accuracy: 0.7349
Epoch 10/300
490/490 [==============================] - 0s 392us/step - l

490/490 [==============================] - 0s 394us/step - loss: 0.0078 - accuracy: 0.7718
Epoch 80/300
490/490 [==============================] - 0s 374us/step - loss: 0.0079 - accuracy: 0.7617
Epoch 81/300
490/490 [==============================] - 0s 374us/step - loss: 0.0078 - accuracy: 0.7511
Epoch 82/300
490/490 [==============================] - 0s 398us/step - loss: 0.0080 - accuracy: 0.7614
Epoch 83/300
490/490 [==============================] - 0s 380us/step - loss: 0.0077 - accuracy: 0.7699
Epoch 84/300
490/490 [==============================] - 0s 380us/step - loss: 0.0079 - accuracy: 0.7673
Epoch 85/300
490/490 [==============================] - 0s 387us/step - loss: 0.0077 - accuracy: 0.7503
Epoch 86/300
490/490 [==============================] - 0s 375us/step - loss: 0.0079 - accuracy: 0.7513
Epoch 87/300
490/490 [==============================] - 0s 383us/step - loss: 0.0077 - accuracy: 0.7542
Epoch 88/300
490/490 [==============================] - 0s 388us/step - loss:

490/490 [==============================] - 0s 387us/step - loss: 0.0077 - accuracy: 0.7770
Epoch 158/300
490/490 [==============================] - 0s 386us/step - loss: 0.0075 - accuracy: 0.7649
Epoch 159/300
490/490 [==============================] - 0s 385us/step - loss: 0.0074 - accuracy: 0.7685
Epoch 160/300
490/490 [==============================] - 0s 386us/step - loss: 0.0073 - accuracy: 0.7792
Epoch 161/300
490/490 [==============================] - 0s 383us/step - loss: 0.0075 - accuracy: 0.7568
Epoch 162/300
490/490 [==============================] - 0s 384us/step - loss: 0.0079 - accuracy: 0.7586
Epoch 163/300
490/490 [==============================] - 0s 384us/step - loss: 0.0079 - accuracy: 0.7631
Epoch 164/300
490/490 [==============================] - 0s 398us/step - loss: 0.0079 - accuracy: 0.7745
Epoch 165/300
490/490 [==============================] - 0s 380us/step - loss: 0.0075 - accuracy: 0.7648
Epoch 166/300
490/490 [==============================] - 0s 377us/ste

490/490 [==============================] - 0s 388us/step - loss: 0.0074 - accuracy: 0.7821
Epoch 236/300
490/490 [==============================] - 0s 384us/step - loss: 0.0076 - accuracy: 0.7621
Epoch 237/300
490/490 [==============================] - 0s 381us/step - loss: 0.0074 - accuracy: 0.7698
Epoch 238/300
490/490 [==============================] - 0s 380us/step - loss: 0.0074 - accuracy: 0.7729
Epoch 239/300
490/490 [==============================] - 0s 381us/step - loss: 0.0076 - accuracy: 0.7661
Epoch 240/300
490/490 [==============================] - 0s 387us/step - loss: 0.0071 - accuracy: 0.7818
Epoch 241/300
490/490 [==============================] - 0s 377us/step - loss: 0.0076 - accuracy: 0.7669
Epoch 242/300
490/490 [==============================] - 0s 385us/step - loss: 0.0076 - accuracy: 0.7710
Epoch 243/300
490/490 [==============================] - 0s 386us/step - loss: 0.0076 - accuracy: 0.7572
Epoch 244/300
490/490 [==============================] - 0s 388us/ste

In [18]:
loss, accuracy = model.evaluate(x_test, y_test)
print('Loss: %.8f, Accuracy: %.2f' % (loss, (accuracy*100)))

39/39 [==============================] - 0s 346us/step - loss: 0.0057 - accuracy: 0.7574
Loss: 0.00571274, Accuracy: 75.74


In [19]:
def rescale_output(y):
    ret = []
    for i in range(len(y)):
        max_val = y_maxes[i]
        min_val = y_mins[i]
        val = y[i]
        rescaled_val = (val * (max_val - min_val)) + min_val
        ret.append(rescaled_val)
    return np.array(ret)

In [20]:
num_tests = len(x_test)
def evaluate_poly(idx=(0,num_tests-1,1)):
    index = x_test.index[idx]
    x_select = np.array(x_test.loc[index, :]).reshape(1, -1)
    y_pred = model.predict(x_select)
    y_pred = rescale_output(y_pred[0])
    a_pred = np.flip(y_pred[:-1])
    d_pred = y_pred[-1]
    
    y_select = np.array(y_test.loc[index, :]).reshape(1, -1)
    y_select = rescale_output(y_select[0])
    a_select = np.flip(y_select[:-1])
    d_select = y_select[-1]
    
    print(d_pred, d_select)
    
    poly_pred = np.poly1d(a_pred)
    poly_select = np.poly1d(a_select)
    
    yp = np.linspace(0, d_pred)
    xp = poly_pred(yp)
    
    ys = np.linspace(0, d_select)
    xs = poly_select(ys)
    
    plt.plot(xs, ys)
    plt.plot(xp, yp)
    plt.xlim([-20,20])
    plt.ylim([-1,39])
    plt.show()
    

interact(evaluate_poly)

Widget Javascript not detected.  It may not be installed or enabled properly.


<function __main__.evaluate_poly(idx=(0, 1223, 1))>

In [149]:
model.save("./models/first_model")

INFO:tensorflow:Assets written to: ./models/first_model/assets


In [ ]:
# First model tried
model = Sequential()
model.add(Dense(12, input_dim=x_dim, activation='relu'))
model.add(Dense(8, activation='relu'))
model.add(Dense(y_dim, activation='linear'))
model.compile(loss='mse', optimizer='adam', metrics=['accuracy'])